<a href="https://colab.research.google.com/github/CodedSpirit-dev/ModeloCovidMX19/blob/main/PTfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):
  File "/home/codedspirit/ModeloCovidMX19/rapidsai-csp-utils/colab/pip-install.py", line 28, in <module>
    if ('K80' not in gpu_name):
TypeError: a bytes-like object is required, not 'str'


In [2]:
!wget https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

--2023-11-06 01:20:36--  https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 201.98.60.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|201.98.60.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17989229 (17M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip.5’

datos_abiertos_covi 100%[===================>]  17.16M  8.38MB/s    in 2.0s    

2023-11-06 01:20:38 (8.38 MB/s) - ‘datos_abiertos_covid19.zip.5’ saved [17989229/17989229]



In [3]:
! unzip /content/datos_abiertos_covid19.zip

/bin/bash: line 1: unzip: command not found


In [4]:
import numpy as np
import pandas as pd


data=pd.read_csv('COVID19MEXICO.csv',low_memory=False)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183124 entries, 0 to 1183123
Data columns (total 40 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   FECHA_ACTUALIZACION    1183124 non-null  object
 1   ID_REGISTRO            1183124 non-null  object
 2   ORIGEN                 1183124 non-null  int64 
 3   SECTOR                 1183124 non-null  int64 
 4   ENTIDAD_UM             1183124 non-null  int64 
 5   SEXO                   1183124 non-null  int64 
 6   ENTIDAD_NAC            1183124 non-null  int64 
 7   ENTIDAD_RES            1183124 non-null  int64 
 8   MUNICIPIO_RES          1183124 non-null  int64 
 9   TIPO_PACIENTE          1183124 non-null  int64 
 10  FECHA_INGRESO          1183124 non-null  object
 11  FECHA_SINTOMAS         1183124 non-null  object
 12  FECHA_DEF              1183124 non-null  object
 13  INTUBADO               1183124 non-null  int64 
 14  NEUMONIA               1183124 non

In [5]:
df=data

df = df.drop(columns=['SECTOR','MUNICIPIO_RES','HABLA_LENGUA_INDIG','ID_REGISTRO','ORIGEN','PAIS_NACIONALIDAD','PAIS_ORIGEN','INDIGENA','ENTIDAD_UM','ENTIDAD_NAC','NACIONALIDAD','ENTIDAD_RES'])
df.columns


df['FECHA_ACTUALIZACION']=pd.to_datetime(df['FECHA_ACTUALIZACION'])
df['FECHA_INGRESO']=pd.to_datetime(df['FECHA_INGRESO'])
df['FECHA_SINTOMAS']=pd.to_datetime(df['FECHA_SINTOMAS'])
df['FECHA_DEF']=pd.to_datetime(df['FECHA_DEF'],errors='coerce')


for col in ['TIPO_PACIENTE']:
  df[col]=df[col].astype(str)


from sklearn.preprocessing import LabelEncoder

le= LabelEncoder()


for col in ['TIPO_PACIENTE','SEXO']:
  df[col]=le.fit_transform(df[col])
# crear caracteristicas apartir de las fechas

df['DIAS_DEFUNCION']=(df['FECHA_ACTUALIZACION']- df['FECHA_DEF']).dt.days
df['DIAS_HOSPITALIZACION']=(df['FECHA_ACTUALIZACION']- df['FECHA_INGRESO']).dt.days
df['DIAS_SINTOMAS']=(df['FECHA_ACTUALIZACION']- df['FECHA_SINTOMAS']).dt.days


# Red NUERONAL REVISAR ESTO
df['DIAS_DEFUNCION'] = df['DIAS_DEFUNCION'].apply(lambda x: 0 if x < 0 else x)



df = df.drop(columns=['FECHA_ACTUALIZACION', 'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF','DIAS_DEFUNCION','MIGRANTE'])

X=df.drop(columns=['CLASIFICACION_FINAL'])
y=df['CLASIFICACION_FINAL']

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183124 entries, 0 to 1183123
Data columns (total 25 columns):
 #   Column                 Non-Null Count    Dtype
---  ------                 --------------    -----
 0   SEXO                   1183124 non-null  int64
 1   TIPO_PACIENTE          1183124 non-null  int64
 2   INTUBADO               1183124 non-null  int64
 3   NEUMONIA               1183124 non-null  int64
 4   EDAD                   1183124 non-null  int64
 5   EMBARAZO               1183124 non-null  int64
 6   DIABETES               1183124 non-null  int64
 7   EPOC                   1183124 non-null  int64
 8   ASMA                   1183124 non-null  int64
 9   INMUSUPR               1183124 non-null  int64
 10  HIPERTENSION           1183124 non-null  int64
 11  OTRA_COM               1183124 non-null  int64
 12  CARDIOVASCULAR         1183124 non-null  int64
 13  OBESIDAD               1183124 non-null  int64
 14  RENAL_CRONICA          1183124 non-null  int64
 15

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf=RandomForestClassifier(random_state=42)
#clf.fit(X_train,y_train)
#y_pred=clf.predict(X_test)

#print(classification_report(y_test,y_pred))


In [9]:
import cuml
from cuml.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf=RandomForestClassifier(random_state=42)
#
X_train=X_train.astype('float32')
y_train=y_train.astype('float32')

clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

print(classification_report(y_test,y_pred))


/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


              precision    recall  f1-score   support

           1       0.95      0.01      0.02      1670
           2       0.00      0.00      0.00         5
           3       1.00      1.00      1.00     83882
           4       0.00      0.00      0.00        29
           5       0.59      0.21      0.32       391
           6       0.94      0.97      0.96     12457
           7       0.99      1.00      0.99    138191

    accuracy                           0.99    236625
   macro avg       0.64      0.46      0.47    236625
weighted avg       0.99      0.99      0.99    236625



/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

In [10]:
import cuml
from cuml.ensemble import RandomForestClassifier
from cuml.neighbors import KNeighborsClassifier


cuml_RFC=RandomForestClassifier(random_state=42)
cuml_KNC=KNeighborsClassifier()

classifiers=[
    {
        'Ramdom Forest',cuml_RFC
    },
    {
        'KNN',cuml_RFC
    }
]


for name,clf in classifiers:
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print(f'Modelo resultados')
  print(classification_report(y_test,y_pred))




/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


Modelo resultados
              precision    recall  f1-score   support

           1       0.95      0.01      0.02      1670
           2       0.00      0.00      0.00         5
           3       1.00      1.00      1.00     83882
           4       0.00      0.00      0.00        29
           5       0.59      0.21      0.32       391
           6       0.94      0.97      0.96     12457
           7       0.99      1.00      0.99    138191

    accuracy                           0.99    236625
   macro avg       0.64      0.46      0.47    236625
weighted avg       0.99      0.99      0.99    236625



/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

Modelo resultados
              precision    recall  f1-score   support

           1       0.95      0.01      0.02      1670
           2       0.00      0.00      0.00         5
           3       1.00      1.00      1.00     83882
           4       0.00      0.00      0.00        29
           5       0.59      0.21      0.32       391
           6       0.94      0.97      0.96     12457
           7       0.99      1.00      0.99    138191

    accuracy                           0.99    236625
   macro avg       0.64      0.46      0.47    236625
weighted avg       0.99      0.99      0.99    236625



/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

In [11]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [12]:
! pip install gradio

In [13]:
def predict(sexo, tipo_paciente, intubado, neumonia, edad, embarazo, diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno,
            migrante, uci, dias_hospitalizacion, dias_sintomas ): 

SyntaxError: incomplete input (898689771.py, line 5)

In [ ]:
pip install typing-extensions --upgrade

Note: you may need to restart the kernel to use updated packages.


In [14]:
import gradio as gr
import pandas as pd
import numpy as np




def predict(sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
             diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno,
           uci, dias_hospitalizacion, dias_sintomas):


    # Crea un DataFrame con los valores de entrada
    input_data = pd.DataFrame([[
        sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
       diabetes, epoc, asma, inmusupr,
        hipertension, otra_com, cardiovascular, obesidad,
        renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
        resultado_lab, toma_muestra_antigeno, resultado_antigeno, uci, dias_hospitalizacion, dias_sintomas
        ]], columns=[
        'SEXO', 'TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'EDAD', 'EMBARAZO','DIABETES', 'EPOC',
        'ASMA', 'INMUSUPR','HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD','RENAL_CRONICA',
        'TABAQUISMO', 'OTRO_CASO', 'TOMA_MUESTRA_LAB','RESULTADO_LAB', 'TOMA_MUESTRA_ANTIGENO',
        'RESULTADO_ANTIGENO','UCI', 'DIAS_HOSPITALIZACION', 'DIAS_SINTOMAS'])

    prediction = clf.predict(input_data)
    return prediction




gr.Row
inputs = [
    gr.Radio(choices=[0, 1], label='Sexo'),
    gr.Radio(choices=[0, 1], label='Tipo de Paciente'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Intubado'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Neumonía'),
    gr.Slider(minimum=0, maximum=120, value=30, label='Edad'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Embarazo'),
    gr.Radio(choices=[1, 2, 98], label='Diabetes'),
    gr.Radio(choices=[1, 2, 98], label='EPOC'),
    gr.Radio(choices=[1, 2, 98], label='Asma'),
    gr.Radio(choices=[1, 2, 98], label='Inmunosupresión'),
    gr.Radio(choices=[1, 2, 98], label='Hipertensión'),
    gr.Radio(choices=[1, 2, 98], label='Otra Comorbilidad'),
    gr.Radio(choices=[1, 2, 98], label='Cardiovascular'),
    gr.Radio(choices=[1, 2, 98], label='Obesidad'),
    gr.Radio(choices=[1, 2, 98], label='Enfermedad Renal Crónica'),
    gr.Radio(choices=[1, 2, 98], label='Tabaquismo'),
    gr.Radio(choices=[1, 2, 99], label='Contacto con otro caso'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de laboratorio'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de laboratorio'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='UCI'),
    gr.Number(label='Días de Hospitalización',value=0),
    gr.Number(label='Días con Síntomas',value=0),
]

# Crear componente de salida para Gradio
outputs = gr.Textbox(label="Predicción")

#! pip install typing-extensions --upgrade
# Crear la interfaz de Gradio
demo = gr.Interface(fn=predict, inputs=inputs, outputs=outputs)

# Ejecutar la aplicación web
if __name__ == "__main__":
    demo.launch(show_api=False,debug=True)

/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
